In [ ]:
import numpy as np
import proplot as pplt
import meds
import fitsio
import glob
import tqdm

In [ ]:
fnames = glob.glob("cat_*.fits")
print(fnames)
d = np.hstack([fitsio.read(f) for f in tqdm.tqdm(fnames)])

In [ ]:
fig, axs = pplt.subplots()
axs.hist(d['MDET_T_RATIO'], log=True)

fig, axs = pplt.subplots()
axs.hist(d['MDET_T'], log=True)

In [ ]:
m = meds.MEDS("DES0420-6039_r5366p01_r_pizza-cutter-slices.fits.fz")
m = meds.MEDS("DES0219-0541_r5366p01_r_pizza-cutter-slices.fits.fz")
m = meds.MEDS("DES0455-5123_r5366p01_r_pizza-cutter-slices.fits.fz")

In [ ]:
msk = (
    (d["TILENAME"] == "DES0455-5123")
    & (d["MDET_T"] > 0.7)
    & (d["MDET_STEP"] == "noshear")
)

print(np.unique(d[msk]["TILENAME"]))

In [ ]:
slice_ids = np.unique(d[msk]["SLICE_ID"])
print(len(slice_ids))

In [ ]:
rc = fitsio.read("DES0455-5123_r5366p01_metadetect.fits.fz")

In [ ]:
import sxdes

for i, slice_id in enumerate(slice_ids):
    
    # get the catalog and seg map using the convenience function
    
    img = m.get_cutout(slice_id, 0)
    wgt = m.get_cutout(slice_id, 0, type="weight")
    ormask = m.get_cutout(slice_id, 0, type="mfrac")
    fig, axs = pplt.subplots(refaspect=1, refwidth=8, nrows=1, ncols=2)
    ax = axs[0]
    ax.imshow(np.arcsinh(img * np.sqrt(wgt)), origin="lower", cmap="rocket")
    smsk = d[msk]["SLICE_ID"] == slice_id
    ax.plot(d[msk]["SLICE_COL"][smsk], d[msk]["SLICE_ROW"][smsk], "b.")
    print(d[msk]["MDET_T"][smsk], d[msk]["PSF_T"][smsk], d[msk]["MDET_S2N"][smsk])
    ax.grid(False)
    ax.format(title=slice_id)

#     cat, _ = sxdes.run_sep(img, 1.0/np.sqrt(wgt))
#     print(cat.shape)
#     ax.plot(cat["x"], cat["y"], "g.")
    
    rmsk = (
        (rc["slice_id"] == slice_id) & 
(
            (rc["mdet_T_ratio"] > 1.2) & (rc["mdet_s2n"] > 15)
        )
    )
    print(np.sum(rmsk))
    print(rc[rmsk]["mdet_T_ratio"])
    ax.plot(rc[rmsk]["slice_col"], rc[rmsk]["slice_row"], "c.")
    
#     omsk = (
#         (d["TILENAME"] == "DES0455-5123")
#         & (d["SLICE_ID"] == slice_id)
#     )
#     print(np.sum(omsk))
#     ax.plot(d[omsk]["SLICE_COL"], d[omsk]["SLICE_ROW"], "b.")
    
    ax = axs[1]
    ax.imshow(np.arcsinh(ormask), cmap="rocket", origin="lower")
    if i == 10:
        break

In [ ]:
ppmsk = rc["flags"] == 0

fig, axs = pplt.subplots(refwidth=6)
axs.hexbin(
    np.log10(rc["mdet_s2n"][ppmsk]), 
    rc["mdet_T_ratio"][ppmsk], 
    bins='log', 
    cmap="rocket",
    extent=[np.log10(5), 3, 0.0, 3.0]
)
axs.hlines(1.1, 0.5, 3)
axs.hlines(1.2, 0.5, 3)

In [ ]:
rc[rmsk]["mdet_T_ratio"]